In [1]:
import datetime
import matplotlib
import numpy as np
import pandas as pd
import pyreadr

import os

In [2]:
USER_KEY = 'userID'
TIME_KEY = 'timestamp'
ITEM_KEY = 'usID'
SESSION_KEY = 'sentenceID'

In [3]:
path_results = '../results/sequence-level/single/drop_seq_onoff_all/untuned/'
# path_results = '../results/drop_on/untuned/'
# path_results = '../results/drop_on-drop_off/untuned/'

test_data = pd.read_hdf('../../data/sequence-level/single/' + 'events-seq_drop_onoff_all.hdf', 'test') # make sure this aligns with path_results

In [4]:
results_files = [f for f in os.listdir(path_results) if 'Saver' not in f]
results = pd.DataFrame()
for file in results_files:
    df = pd.read_csv(path_results + file, sep = ';')
    # TBD: extract window number from file name, create new column
    df.drop(['Metrics', 'Saver@50: '], axis=1, inplace=True)
    df.drop(df.filter(regex='Unnamed'), axis=1, inplace=True) # drop 'Unnamed: 24' column containing only NaNs
    df.rename(columns = lambda x : str(x)[:-2], inplace=True) # remove colon and whitespace from all column names
    df.insert(0, 'model', "_".join(file.split('_')[2:-1]))
    results = results.append(df)

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: '../results/sequence-level/single/drop_seq_onoff_all/untuned/'

In [ ]:
results

### extracting predictions

In [ ]:
# create app and user mappings
data = pd.read_csv('../../data/sequence-level/data_seq.csv')

mapping = dict([(y,x+1) for x,y in enumerate(sorted(set(data['category_list'])))])
mapping_reverse = dict((v,k) for k,v in mapping.items())

In [6]:
mappping

NameError: name 'mappping' is not defined

In [73]:
# extract ground truth from test data (test_data) for a single item (position) in a single session (sessionId)
def extract_ground_truth(ID, position, test_data):
    relevant_df = test_data[test_data[SESSION_KEY]==ID]
    index = relevant_df.index[position+1]
    ground_truth = relevant_df[ITEM_KEY][index]
    return ground_truth

In [74]:
predictions_files = [f for f in os.listdir(path_results) if 'Saver' in f]
predictions = pd.DataFrame()
for file in predictions_files:
    model = "_".join(file.split('_')[2:-1])
    df = pd.read_csv(path_results + file, sep = ';')
    if 'sessionID' not in predictions.columns:
        predictions['sessionID'] = df['SessionId']
    if 'position' not in predictions.columns:
        predictions['position'] = df['Position']
    if 'ground_truth' not in predictions.columns:
        predictions['ground_truth'] = predictions.apply(lambda x: extract_ground_truth(x['sessionID'], x['position'], test_data), axis=1)
        predictions['ground_truth_name'] = predictions['ground_truth'].apply(lambda x: mapping_reverse[x])
    predictions['recs-' + model] = df['Recommendations'].apply(lambda x: [int(i) for i in x.split(',')])
    predictions['recs_names-' + model] = predictions['recs-' + model].apply(lambda x: [mapping_reverse[i] for i in x])
    predictions['scores-' + model] = df['Scores'].apply(lambda x: x.split(','))

In [75]:
def print_predictions(predictions, sessionID, num_recs, models):
    # predictions must contain columns named 'sessionID' and 'position', containing the respective values
    predictions_dict = {}
    for pos in positions:
        row = predictions[(predictions.sessionID == sessionID) & (predictions.position == pos)]
        ground_truth = row.ground_truth_name.to_string(index=False)
#         print('sessionID: ' + str(sessionID) + ', position: ' + str(pos))
#         print('ground truth: ' + str(row.ground_truth_name.to_string(index=False)))
        df = pd.DataFrame()
        for model in models:
            df[model] = [row['recs_names-' + model].tolist()[0][i] for i in range(num_recs)]
        name = str(sessionID) + '_' + str(pos)
        predictions_dict[name] = (sessionID, pos, ground_truth, df)
    return predictions_dict

In [76]:
sessionID = predictions.sessionID.unique()[0]
num_recs = 5
positions = predictions.position[predictions.sessionID==sessionID]
models = results.model.tolist()

In [77]:
print_predictions(predictions, sessionID, num_recs, models)

{'27_0': (27,
  0,
  'ON,Launcher,Orientation,Launcher,Financial,Laun...',
                                                    ar  \
  0  ON,Internet_Browser,ON,Launcher,Internet_Brows...   
  1  ON,Launcher,Messaging,Launcher,TVVideo_Apps,La...   
  2  ON,Launcher,Social_Networks,Launcher,Settings,...   
  3  ON,Launcher,Orientation,Launcher,Financial,Lau...   
  4  ON,Launcher,ON,Messaging,Timer_Clocks,Messagin...   
  
                                      gru4rec_Reminder  \
  0  ON,Internet_Browser,ON,Launcher,Internet_Brows...   
  1  ON,Launcher,Messaging,Launcher,TVVideo_Apps,La...   
  2  ON,Music_Audio_Radio,PLAY,Launcher,Messaging,L...   
  3  ON,Launcher,ON,Messaging,Launcher,Social_Netwo...   
  4  ON,Music_Audio_Radio,PAUSE,PLAY,PAUSE,PLAY,Lau...   
  
                                               gru4rec  \
  0  ON,Music_Audio_Radio,PLAY,Launcher,Messaging,L...   
  1  ON,Launcher,ON,Messaging,Launcher,Social_Netwo...   
  2  ON,Financial,ON,Financial,Launcher,Messaging

In [78]:
# predictions.apply(lambda x: x['recs-sr_B'][0] == x['ground_truth'], axis=1).sum()/len(predictions)

### performance by position

We expect performance of neural algorithms to increase as the position increases. Since this is sequence-level analysis, there are plenty of long sentences.

In [79]:
algorithms = [i for i in predictions.columns if i.startswith('recs-')]

In [80]:
# helper function outputting whether ground truth is in recommendation list of length k for a single algorithm and item
def calc_hr_k(ground_truth, rec_list, k):
    return ground_truth in rec_list[:k]

In [81]:
pos_counts = predictions['position'].value_counts()

In [174]:
k=10

In [175]:
perf_by_pos = pd.DataFrame()
for pos in pos_counts.index:
    pred_pos = predictions[predictions['position']==pos]
    df = pd.DataFrame()
    df['position'] = [pos]
    count = pos_counts[pos]
    df['count'] = [count]
    for algo in algorithms:
        algo_name = ''.join(algo.split('-')[1:])
        value = pred_pos.apply(lambda x: calc_hr_k(x['ground_truth'], x[algo], k), axis=1).sum()/len(pred_pos)
        df[algo_name] = [value]
    perf_by_pos = perf_by_pos.append(df).reset_index(drop=True)

In [176]:
perf_by_pos.head(20)

,position,count,ar,gru4rec_Reminder,gru4rec,hgru4rec,shan,sknn,sr_BR,sr,stan_ER,stan,vsknn_EBR,vsknn,vstan_EBR,vstan
0,0,277,0.490975,0.418773,0.314079,0.223827,0.440433,0.519856,0.534296,0.509025,0.501805,0.480144,0.364621,0.129964,0.292419,0.487365
1,1,262,0.461832,0.370229,0.251908,0.183206,0.408397,0.450382,0.469466,0.469466,0.477099,0.473282,0.293893,0.137405,0.282443,0.480916
2,2,249,0.502008,0.437751,0.317269,0.188755,0.433735,0.530120,0.510040,0.530120,0.522088,0.538153,0.281124,0.176707,0.289157,0.554217
3,3,241,0.514523,0.448133,0.331950,0.199170,0.402490,0.502075,0.526971,0.510373,0.510373,0.506224,0.253112,0.149378,0.269710,0.510373
4,4,232,0.538793,0.413793,0.323276,0.206897,0.413793,0.491379,0.504310,0.551724,0.512931,0.504310,0.262931,0.150862,0.232759,0.525862
5,5,226,0.526549,0.438053,0.340708,0.185841,0.438053,0.539823,0.530973,0.522124,0.530973,0.530973,0.243363,0.154867,0.221239,0.535398
6,6,220,0.500000,0.490909,0.322727,0.200000,0.413636,0.513636,0.513636,0.545455,0.486364,0.504545,0.254545,0.154545,0.190909,0.518182
7,7,214,0.495327,0.476636,0.383178,0.280374,0.350467,0.481308,0.518692,0.556075,0.462617,0.500000,0.228972,0.149533,0.224299,0.495327
8,8,209,0.502392,0.397129,0.315789,0.167464,0.411483,0.497608,0.497608,0.526316,0.507177,0.483254,0.263158,0.148325,0.229665,0.497608
9,9,203,0.541872,0.517241,0.384236,0.270936,0.389163,0.522167,0.532020,0.556650,0.536946,0.541872,0.280788,0.182266,0.275862,0.541872


In [181]:
# considering only positions <= vs. >
perf_by_pos = pd.DataFrame()
cutoff = 5
for pos in ['<=', '>']:
    if pos=='<=':
        pred_pos = predictions[predictions['position']<=cutoff]
    else:
        pred_pos = predictions[predictions['position']>cutoff]
    df = pd.DataFrame()
    df['position'] = [pos]
    for algo in algorithms:
        algo_name = ''.join(algo.split('-')[1:])
        value = pred_pos.apply(lambda x: calc_hr_k(x['ground_truth'], x[algo], k), axis=1).sum()/len(pred_pos)
        df[algo_name] = [value]
    perf_by_pos = perf_by_pos.append(df).reset_index(drop=True)

In [182]:
perf_by_pos

,position,ar,gru4rec_Reminder,gru4rec,hgru4rec,shan,sknn,sr_BR,sr,stan_ER,stan,vsknn_EBR,vsknn,vstan_EBR,vstan
0,<=,0.504371,0.420309,0.312038,0.198386,0.422999,0.505044,0.512441,0.514459,0.508406,0.504371,0.285810,0.149294,0.266308,0.514459
1,>,0.569509,0.537492,0.403616,0.258792,0.448905,0.541971,0.567518,0.599536,0.559721,0.566025,0.260783,0.151792,0.229098,0.576808
